# Part 0

## Import Libs

In [1]:
!pip -q install D:\\projects\\Kaggle\\CMI24\\pytorch_tabnet-4.1.0-py3-none-any.whl

In [2]:
import torch
import numpy as np
import pandas as pd
import random
import os
import seaborn as sns
# import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
# import polars as pl
# import polars.selectors as cs
import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
# import seaborn as sns

from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# from keras.models import Model
# from keras.layers import Input, Dense
# from keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline

from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split
from pytorch_tabnet.callbacks import Callback
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

## Config

In [3]:
is_debug = False

class CFG:
    
    n_splits = 5
    
    seed = 308
    
#     root_dir = '/kaggle/input/child-mind-institute-problematic-internet-use/'
    example_dir = 'D:\\projects\\Kaggle\\CMI24\\dataset\\child-mind-institute-problematic-internet-use\\sample_submission.csv'
    
    #meta_data:
    train_meta_dir = 'D:\\projects\\Kaggle\\CMI24\\dataset\\child-mind-institute-problematic-internet-use\\train.csv'
    test_meta_dir = 'D:\\projects\\Kaggle\\CMI24\\dataset\\child-mind-institute-problematic-internet-use\\test.csv'
    
    #time_series:
    train_ts_dir = 'D:\\projects\\Kaggle\\CMI24\\dataset\\child-mind-institute-problematic-internet-use\\series_train.parquet'
    tets_ts_dir = 'D:\\projects\\Kaggle\CMI24\\dataset\\child-mind-institute-problematic-internet-use\\series_test.parquet'


In [4]:

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 
          'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']


LGB_Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,  # Increased from 6.59
    'lambda_l2': 0.01,  # Increased from 2.68e-06
    'random_state': CFG.seed,
    'device': 'gpu'

}

XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,  # Increased from 0.1
    'reg_lambda': 5,  # Increased from 1
    'random_state': CFG.seed,
    # 'tree_method': 'exact'
    'tree_method': 'gpu_hist'
}

CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': CFG.seed,
    'cat_features': cat_c,
    'verbose': 0,
    'l2_leaf_reg': 10 ,
    'task_type': 'GPU'
}

## Set Random

In [5]:
def set_seed(seed=308):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(CFG.seed) 

## Utils

In [6]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# 1. 基本统计特征
def compute_basic_statistics(df):
    return df.describe().values.reshape(-1)

# 2. 差分特征
def compute_difference_features(df, order=1):
    diff_features = []
    for col in df.columns:
        for i in range(1, order + 1):
            diff_col = df[col].diff(i).dropna()
            diff_features.append(diff_col.describe().values)
    return np.concatenate(diff_features)

# 3. 频域特征
def compute_frequency_features(df):
    freq_features = []
    for col in df.columns:
        fft_vals = np.fft.fft(df[col].dropna())
        fft_mag = np.abs(fft_vals)
        freq_features.append([np.mean(fft_mag), np.std(fft_mag), np.max(fft_mag)])
    return np.array(freq_features).reshape(-1)

# 4. 滞后特征
def compute_lag_features(df, lags=3):
    lag_features = []
    for col in df.columns:
        for lag in range(1, lags + 1):
            lag_col = df[col].shift(lag).dropna()
            lag_features.append(lag_col.describe().values)
    return np.concatenate(lag_features)

# 主处理函数，选择不同的方法
def process_file(filename, dirname, methods):
    file_path = os.path.join(dirname, filename, 'part-0.parquet')
    try:
        df = pd.read_parquet(file_path)
        df.drop('step', axis=1, inplace=True, errors='ignore')
        
        features = []
        if 'basic' in methods:
            features.append(compute_basic_statistics(df))
        if 'difference' in methods:
            features.append(compute_difference_features(df))
        if 'frequency' in methods:
            features.append(compute_frequency_features(df))
        if 'lag' in methods:
            features.append(compute_lag_features(df))
        
        # 将所有特征拼接为一维数组
        all_features = np.concatenate(features)
        file_id = filename.split('=')[1]
        
        return all_features, file_id
    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        return None

# 加载所有文件并提取特征
def load_time_series(dirname, methods) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname, methods), ids), total=len(ids)))
    
    results = [res for res in results if res is not None]
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"feature_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

# 示例使用
methods = ['basic', 'difference']#, 'frequency', 'lag'
# df = load_time_series("your_directory", methods)
# print(df.head())


In [ ]:
# load data：

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim, hidden_dims=None):
        super(AutoEncoder, self).__init__()
        
        # 编码器层
        encoder_layers = []
        current_dim = input_dim
        if hidden_dims:
            for hidden_dim in hidden_dims:
                encoder_layers.append(nn.Linear(current_dim, hidden_dim))
                encoder_layers.append(nn.BatchNorm1d(hidden_dim))
                encoder_layers.append(nn.LeakyReLU())
                current_dim = hidden_dim
        encoder_layers.append(nn.Linear(current_dim, encoding_dim))
        encoder_layers.append(nn.ReLU())
        self.encoder = nn.Sequential(*encoder_layers)
        
        # 解码器层
        decoder_layers = []
        current_dim = encoding_dim
        for hidden_dim in reversed(hidden_dims or []):
            decoder_layers.append(nn.Linear(current_dim, hidden_dim))
            decoder_layers.append(nn.BatchNorm1d(hidden_dim))
            decoder_layers.append(nn.LeakyReLU())
            current_dim = hidden_dim
        decoder_layers.append(nn.Linear(current_dim, input_dim))
        decoder_layers.append(nn.Sigmoid())
        self.decoder = nn.Sequential(*decoder_layers)
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def perform_autoencoder(df, encoding_dim=50, hidden_dims=[128, 64], epochs=300, batch_size=64, patience=15):

    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim, hidden_dims)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
    
    # 早停机制和保存最优模型
    best_loss = np.inf
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        epoch_loss /= (len(data_tensor) / batch_size)
        
        # 输出损失
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}')
        
        # 早停机制：如果损失没有下降，增加计数器
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            patience_counter = 0
            best_model_state = autoencoder.state_dict()  # 保存最佳模型
        else:
            patience_counter += 1
        
        # 如果超过耐心次数，停止训练
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break
    
    # 加载最佳模型
    if best_model_state:
        autoencoder.load_state_dict(best_model_state)
    
    # 提取编码后的数据
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded




# feature engineering：
def feature_engineering(df):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

# calculate：
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# set cols:
def update(df):
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

  
# trainer
def TrainML(model_class, test_data):
    
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), CFG.n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=CFG.n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead') # Nelder-Mead | # Powell
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission


# sub
def majority_vote(row):
    return row.mode()[0]

# Part 1

## Meta Data

In [8]:
train_meta = pd.read_csv(CFG.train_meta_dir)
test_meta = pd.read_csv(CFG.test_meta_dir)
sample = pd.read_csv(CFG.example_dir)

## Time Series

In [9]:
train_ts = load_time_series(CFG.train_ts_dir, methods) #directly load&process  
test_ts = load_time_series(CFG.tets_ts_dir, methods)
train_ts.shape

100%|██████████| 2/2 [00:00<00:00,  8.82it/s]


(996, 193)

## Merged

In [10]:
df_train = train_ts.drop('id', axis=1)
df_test = test_ts.drop('id', axis=1)

train_ts_encoded = perform_autoencoder(df_train, encoding_dim=180, epochs=100, batch_size=32)
test_ts_encoded = perform_autoencoder(df_test, encoding_dim=180, epochs=100, batch_size=32)

time_series_cols = train_ts_encoded.columns.tolist()
train_ts_encoded["id"]=train_ts["id"]
test_ts_encoded['id']=test_ts["id"]

train = pd.merge(train_meta, train_ts_encoded, how="left", on='id')
test = pd.merge(test_meta, test_ts_encoded, how="left", on='id')

imputer = KNNImputer(n_neighbors=5)
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
imputed_data = imputer.fit_transform(train[numeric_cols])
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)
train_imputed['sii'] = train_imputed['sii'].round().astype(int)
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]
        
train = train_imputed

train = feature_engineering(train)
train = train.dropna(thresh=10, axis=0)
test = feature_engineering(test)

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train_meta, train_ts, how="left", on='id')
test = pd.merge(test_meta, test_ts, how="left", on='id')

Epoch [10/100], Loss: 0.6346]
Epoch [20/100], Loss: 0.6346]
Epoch [30/100], Loss: 0.6346]
Epoch [40/100], Loss: 0.6346]
Epoch [50/100], Loss: 0.6346]
Epoch [60/100], Loss: 0.6346]
Epoch [70/100], Loss: 0.6346]
Epoch [80/100], Loss: 0.6346]
Epoch [90/100], Loss: 0.6346]
Epoch [100/100], Loss: 0.6346]
Epoch [10/100], Loss: 0.5723]
Epoch [20/100], Loss: 0.3984]
Epoch [30/100], Loss: 0.3984]
Epoch [40/100], Loss: 0.3984]
Epoch [50/100], Loss: 0.3984]
Epoch [60/100], Loss: 0.3984]
Epoch [70/100], Loss: 0.3984]
Epoch [80/100], Loss: 0.3984]
Epoch [90/100], Loss: 0.3984]
Epoch [100/100], Loss: 0.3984]


In [11]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset='sii')

train = update(train)
test = update(test)

"""This Mapping Works Fine For me I also Check Each Values in Train and test Using Logic. There no Data Lekage."""
for col in cat_c:
    mapping_train = create_mapping(col, train)
    mapping_test = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping_train).astype(int)
    test[col] = test[col].replace(mapping_test).astype(int)
    
print(f'Train Shape : {train.shape} || Test Shape : {test.shape}')

Train Shape : (2736, 251) || Test Shape : (20, 250)


# Part 2

## Model

In [12]:
class TabNetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **kwargs):
        self.model = TabNetRegressor(**kwargs)
        self.kwargs = kwargs
        self.imputer = SimpleImputer(strategy='median')
        self.best_model_path = 'best_tabnet_model.pt'
        
    def fit(self, X, y):
        # Handle missing values
        X_imputed = self.imputer.fit_transform(X)
        
        if hasattr(y, 'values'):
            y = y.values
            
        # Create internal validation set
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_imputed, 
            y, 
            test_size=0.2,
            random_state=42
        )
        
        # Train TabNet model
        history = self.model.fit(
            X_train=X_train,
            y_train=y_train.reshape(-1, 1),
            eval_set=[(X_valid, y_valid.reshape(-1, 1))],
            eval_name=['valid'],
            eval_metric=['mse'],
            max_epochs=500,
            patience=50,
            batch_size=1024,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False,
            callbacks=[
                TabNetPretrainedModelCheckpoint(
                    filepath=self.best_model_path,
                    monitor='valid_mse',
                    mode='min',
                    save_best_only=True,
                    verbose=True
                )
            ]
        )
        
        # Load the best model
        if os.path.exists(self.best_model_path):
            self.model.load_model(self.best_model_path)
            os.remove(self.best_model_path)  # Remove temporary file
        
        return self
    
    def predict(self, X):
        X_imputed = self.imputer.transform(X)
        return self.model.predict(X_imputed).flatten()
    
    def __deepcopy__(self, memo):
        # Add deepcopy support for scikit-learn
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, deepcopy(v, memo))
        return result
    
# TabNet hyperparameters
TabNet_Params = {
    'n_d': 64,              # Width of the decision prediction layer
    'n_a': 64,              # Width of the attention embedding for each step
    'n_steps': 5,           # Number of steps in the architecture
    'gamma': 1.5,           # Coefficient for feature selection regularization
    'n_independent': 2,     # Number of independent GLU layer in each GLU block
    'n_shared': 2,          # Number of shared GLU layer in each GLU block
    'lambda_sparse': 1e-4,  # Sparsity regularization
    'optimizer_fn': torch.optim.Adam,
    'optimizer_params': dict(lr=2e-2, weight_decay=1e-5),
    'mask_type': 'entmax',
    'scheduler_params': dict(mode="min", patience=10, min_lr=1e-5, factor=0.5),
    'scheduler_fn': torch.optim.lr_scheduler.ReduceLROnPlateau,
    'verbose': 1,
    'device_name': 'cuda' if torch.cuda.is_available() else 'cpu'
}

class TabNetPretrainedModelCheckpoint(Callback):
    def __init__(self, filepath, monitor='val_loss', mode='min', 
                 save_best_only=True, verbose=1):
        super().__init__()  # Initialize parent class
        self.filepath = filepath
        self.monitor = monitor
        self.mode = mode
        self.save_best_only = save_best_only
        self.verbose = verbose
        self.best = float('inf') if mode == 'min' else -float('inf')
        
    def on_train_begin(self, logs=None):
        self.model = self.trainer  # Use trainer itself as model
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return
        # Check if current metric is better than best
        if (self.mode == 'min' and current < self.best) or \
           (self.mode == 'max' and current > self.best):
            if self.verbose:
                print(f'\nEpoch {epoch}: {self.monitor} improved from {self.best:.4f} to {current:.4f}')
            self.best = current
            if self.save_best_only:
                self.model.save_model(self.filepath)  # Save the entire model

## Train

In [14]:
# ensemble 1
LGB_Model = LGBMRegressor(**LGB_Params, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)
TabNet_Model = TabNetWrapper(**TabNet_Params) 

voting_model = VotingRegressor(estimators=[
    ('lightgbm', LGB_Model),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model),
    ('tabnet', TabNet_Model)
])  # CV:

Submission1 = TrainML(voting_model, test)    

Training Folds: 100%|██████████| 5/5 [05:13<00:00, 62.72s/it]

Mean Train QWK --> 0.6904
Mean Validation QWK ---> 0.3596
----> || Optimized QWK SCORE ::  0.457


In [15]:
# ensemble 2



In [16]:
# ensemble 3
imputer = SimpleImputer(strategy='median')

ensemble = VotingRegressor(estimators=[
    ('lgb', Pipeline(steps=[('imputer', imputer), ('regressor', LGBMRegressor(random_state= CFG.seed))])),
    ('xgb', Pipeline(steps=[('imputer', imputer), ('regressor', XGBRegressor(random_state= CFG.seed))])),
    ('cat', Pipeline(steps=[('imputer', imputer), ('regressor', CatBoostRegressor(random_state= CFG.seed, silent=True))])),
    ('rf', Pipeline(steps=[('imputer', imputer), ('regressor', RandomForestRegressor(random_state= CFG.seed))])),
    ('gb', Pipeline(steps=[('imputer', imputer), ('regressor', GradientBoostingRegressor(random_state= CFG.seed))])),
    ('tabnet', Pipeline(steps=[('imputer', imputer), ('regressor', TabNetWrapper(**TabNet_Params))])) 
])

Submission2 = TrainML(ensemble, test)

Training Folds: 100%|██████████| 5/5 [05:20<00:00, 64.10s/it]

Mean Train QWK --> 0.8503
Mean Validation QWK ---> 0.3509


----> || Optimized QWK SCORE ::  0.460


## Sub

In [17]:
sub1 = Submission1
sub2 = Submission2
# sub3 = Submission3

sub1 = sub1.sort_values(by='id').reset_index(drop=True)
sub2 = sub2.sort_values(by='id').reset_index(drop=True)
# sub3 = sub3.sort_values(by='id').reset_index(drop=True)

combined = pd.DataFrame({
    'id': sub1['id'],
    'sii_1': sub1['sii'],
    'sii_2': sub2['sii']#,
    # 'sii_3': sub3['sii']
})

def majority_vote(row):
    return row.mode()[0]

combined['final_sii'] = combined[['sii_1', 'sii_2']].apply(majority_vote, axis=1)  #, 'sii_3'

final_submission = combined[['id', 'final_sii']].rename(columns={'final_sii': 'sii'})

final_submission.to_csv('submission.csv', index=False)

# Submission1.to_csv('submission.csv', index=False)
# Submission1

print("Majority voting completed and saved to 'Final_Submission.csv'")


Majority voting completed and saved to 'Final_Submission.csv'


In [18]:
final_submission
# # print(Submission['sii'].value_counts())

,id,sii
0,00008ff9,1
1,000fd460,0
2,00105258,0
3,00115b9f,0
4,0016bb22,1
5,001f3379,1
6,0038ba98,0
7,0068a485,0
8,0069fbed,1
9,0083e397,0
